In [1]:
# !pip install --trusted-host pypi.org --trusted-host pypi.pyhton.org --trusted-host files.pythonhosted.org -U --upgrade xlrd


In [127]:
import os
import sys 
import pandas as pd
import re
from selenium import webdriver 
import win32com.client
import time
from datetime import date
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import xlsxwriter
import concurrent.futures
import datetime
import requests

import pdfplumber

import matplotlib.pyplot as plt
import plotly.express as px

import shutil

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
#from bs4 import BeautifulSoup
import PyPDF2
from PyPDF2 import PdfReader

import xlrd

from dateutil.relativedelta import relativedelta

In [3]:
today = date.today().strftime("%Y-%m-%d")


In [4]:
def start_driver():

    options = webdriver.ChromeOptions()

    #Select desired download folder.
    download_directory = "C:\\Users\\bd1266\\Downloads\\Data Download"
    prefs = {
                "download.default_directory": download_directory,
                 "download.prompt_for_download": False,
                 "download.directory_upgrade": True,
                 "safebrowsing.enabled": True,
                 "safebrowsing.disable_download_protection": True}
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver



# Economia Local 

## 1- INDEC

In [45]:
try:
    driver.quit()
except:
    print("no driver open")       

In [46]:
driver = start_driver()

In [47]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/143/distribution/143.3/download/emae-valores-anuales-indice-base-2004-mensual.csv")
time.sleep(12)

In [48]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "emae" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'emae' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

Renamed 'emae-valores-anuales-indice-base-2004-mensual.csv' to '2025-01-20_emae-valores-anuales-indice-base-2004-mensual.csv'.


In [49]:
emae_df = pd.read_csv(new_file_path)

In [50]:
emae_df

,indice_tiempo,emae_original,emae_desestacionalizada,emae_tendencia_ciclo,emae_original_via,emae_desestacionalizada_vm,emae_tendencia_ciclo_vm
0,2004-01-01,92.627506,98.448943,96.046952,NaN,NaN,NaN
1,2004-02-01,90.186179,98.177010,96.646226,NaN,-0.002762,0.006239
2,2004-03-01,101.883298,97.807535,97.316219,NaN,-0.003763,0.006932
3,2004-04-01,102.567430,95.042873,98.013886,NaN,-0.028266,0.007169
4,2004-05-01,109.877504,96.892728,98.738458,NaN,0.019463,0.007393
...,...,...,...,...,...,...,...
245,2024-06-01,145.425842,140.156246,142.546924,-0.041023,-0.000349,0.000936
246,2024-07-01,148.149235,144.368779,142.766561,-0.005390,0.030056,0.001541
247,2024-08-01,145.874373,145.634132,143.031888,-0.032202,0.008765,0.001858
248,2024-09-01,143.815002,145.611025,143.302431,-0.025299,-0.000159,0.001891


In [51]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

Moved '2025-01-20_emae-valores-anuales-indice-base-2004-mensual.csv' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.


## Descarga de texto 

### Texto pdf IPC - pagina 3

##### Intentamos extrayendo el texto desde el pdf del indec pero es mucho más difícil.
##### Igualmente dejamos el codigo aca en caso de que se quiera guardar el analisis de texto.

In [135]:
try:
    driver.quit()
except:
    print("no driver open")   

In [138]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.indec.gob.ar/")
time.sleep(12)

In [139]:
time.sleep(5)
button_xpath = '/html/body/div[2]/div[1]/div[2]/div[1]/div/div/div/div/div[3]/a/div/div[2]'  # Replace with your button's XPath
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, button_xpath))
).click()
time.sleep(2) 

In [140]:
original_window = driver.current_window_handle

In [141]:
# Just in case the browser doesn't automatically switch windows.

WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
for handle in driver.window_handles:
    if handle != original_window:
        driver.switch_to.window(handle)
        break
time.sleep(1)
pdf_url = driver.current_url  

In [142]:
pdf_url

'https://www.indec.gob.ar/uploads/informesdeprensa/ipc_01_2517A7124C09.pdf'

In [56]:
response = requests.get(pdf_url)
with open('document.pdf', 'wb') as f:
    f.write(response.content)

In [143]:
with open('document.pdf', 'rb') as f:
    pdf_reader = PdfReader(f)
    if len(pdf_reader.pages) >= 3: 
        page3 = pdf_reader.pages[2]  
        
        text_page3 = page3.extract_text()  
        print("Text from page 3:", text_page3)
    else:
        print("The PDF does not have enough pages.")

Text from page 3: Fuente: INDEC, Dirección Nacional de Estadísticas de Precios. Dirección de Índices de Precios de Consumo.Índices de precios Buenos Aires, 12 de noviembre de 2024
Octubre de 2024. Resumen ejecutivoÍndice de precios al consumidor
%
Fuente:  INDEC, Dirección Nacional de Estadísticas de Precios. Dirección de Índices de Precios de Consumo. Octubre de 2024 - Variación mensual de las 12 divisiones del IPC. Total nacional
 Menor variación porcentual  Mayor variación porcentual
Destacados del mes
El Nivel general del Índice de precios al consumidor registró un alza mensual de 2,7% en octubre de 2024, y acumuló en el 
año una variación de 107,0%. En la comparación interanual, el incremento alcanzó el 193,0%.
La división de mayor aumento en el mes fue Vivienda, agua, electricidad, gas y otros combustibles (5,4%) por las 
subas en Alquiler de la vivienda y gastos conexos; Electricidad, gas y otros combustibles; y Suministro de agua, seguida de 
Prendas de vestir y calzado (4,4%).

In [147]:
def extract_content(text):
    # Split the text into lines
    lines = text.splitlines()
    content = []
    recording = False
    
    for line in lines:
        # Check if we should start recording
        if "Destacados del mes" in line:
            recording = True
        
        # If we are recording and line is not just numbers or empty
        if recording:
            # Check if the line contains only numbers or is empty
            if re.match(r'^\d+$', line.strip()):
                break  # Stop recording if we hit a line with only numbers
            
            content.append(line.strip())  # Add non-empty lines to content

    return "\n".join(content)

# Extract and print the content
extracted_text_indec_ipc_ = extract_content(text_page3)

In [149]:
print(extracted_text_indec_ipc_)

Destacados del mes
El Nivel general del Índice de precios al consumidor registró un alza mensual de 2,7% en octubre de 2024, y acumuló en el
año una variación de 107,0%. En la comparación interanual, el incremento alcanzó el 193,0%.
La división de mayor aumento en el mes fue Vivienda, agua, electricidad, gas y otros combustibles (5,4%) por las
subas en Alquiler de la vivienda y gastos conexos; Electricidad, gas y otros combustibles; y Suministro de agua, seguida de
Prendas de vestir y calzado (4,4%).
La división con mayor incidencia fue Restaurantes y hoteles (4,3%) en las regiones GBA y Noroeste, mientras que en
Noreste, Cuyo y Patagonia, fue Vivienda, agua, electricidad, gas y otros combustibles (5,4%). Por último, en Pampeana,
la mayor incidencia se presentó en Alimentos y bebidas no alcohólicas (1,2%), donde se destacaron los aumentos de
Pan y Cereales; Carnes y derivados; y Frutas.
Las dos divisiones que registraron las menores variaciones en octubre de 2024 fueron Transporte (1,2

### Texto pdf Estimador de Actividad Económica - pagina 3

In [162]:
try:
    driver.quit()
except:
    print("no driver open")   

In [163]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.indec.gob.ar/")
time.sleep(12)

In [164]:
time.sleep(5)
button_xpath = '/html/body/div[2]/div[1]/div[2]/div[1]/div/div/div/div/div[5]/a/div/div[1]'  # Replace with your button's XPath
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, button_xpath))
).click()
time.sleep(2) 

In [165]:
original_window = driver.current_window_handle

In [166]:
# Just in case the browser doesn't automatically switch windows.

WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
for handle in driver.window_handles:
    if handle != original_window:
        driver.switch_to.window(handle)
        break
time.sleep(1)
pdf_url = driver.current_url  

In [169]:
response = requests.get(pdf_url)
with open('document.pdf', 'wb') as f:
    f.write(response.content)

In [171]:
with open('document.pdf', 'rb') as f:
    pdf_reader = PdfReader(f)
    if len(pdf_reader.pages) >= 3: 
        page_emae3 = pdf_reader.pages[2]  
        
        text_page_emae3 = page_emae3.extract_text()  
        print("Text from page 3:", text_page_emae3)
    else:
        print("The PDF does not have enough pages.")

Text from page 3: Cuentas nacionales Buenos Aires, 20 de diciembre de 2024
Estimador mensual de actividad económica
Fuente: INDEC, Dirección Nacional de Cuentas Nacionales.Estimación preliminar de octubre de 2024. Resumen ejecutivo
-0,7%
 0,6%
 0,2%
EMAE
Variación %
interanualVariación %
desestacionalizadaVariación %
tendencia-ciclo
Respecto a igual
mes del año
anteriorRespecto al
mes anteriorRespecto al
mes anterior
Destacados del mes
2021
Original Desestacionalizado Tendencia-ciclo
120125130135140145150155160165170
 OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
SEP
OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
SEP
OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
SEP
OCT
120125130135140145150155160165AGO
SEP
OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
SEP
OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
SEP
OCT
NOV
DIC
ENE
FEB
MAR
ABR
MAY
JUN
JUL
AGO
En octubre de 2024, el estimador mensual de actividad económica (EMAE) registró una 
caída de 0,7% en la comparación interanual (ia) y un crecimiento 

In [177]:
def extract_sentences_with_words(text):
    # Split the text into lines.
    lines = text.splitlines()
    content = []
    
    for line in lines:
        # Count words in the line.
        word_count = len(re.findall(r'\S+', line))  # Count non-whitespace tokens.
        
        # If the line has more than five words.
        if word_count > 5:
            content.append(line.strip())  # Add it to the result.

    return "\n".join(content)

# Extract and print the content.

# Extract and print the content.
extracted_content_emae_page3 = extract_sentences_with_words(text_page_emae3)
print(extracted_content_emae_page3)

Cuentas nacionales Buenos Aires, 20 de diciembre de 2024
Fuente: INDEC, Dirección Nacional de Cuentas Nacionales.Estimación preliminar de octubre de 2024. Resumen ejecutivo
En octubre de 2024, el estimador mensual de actividad económica (EMAE) registró una
caída de 0,7% en la comparación interanual (ia) y un crecimiento de 0,6% respecto a septiembre en la medición desestacionalizada (s.e.).
Con relación a igual mes de 2023, cinco sectores de actividad que conforman el EMAE
registraron subas en octubre, entre los que se destacan Explotación de minas y canteras (+7,4% ia) y Agricultura, ganadería, caza y silvicultura (+2,3% ia).
La actividad Explotación de minas y canteras (+7,4% ia) fue, a su vez, la de mayor
incidencia positiva en la variación interanual del EMAE, seguida por Agricultura, ganadería, caza y silvicultura (+2,3% ia) e Intermediación ﬁnanciera (1,8% ia).
Por su parte, diez sectores de actividad registraron caídas en la comparación interanual,
entre los que se destacan Pesc

## 4- Cuentas Nacionales

### 4.1 PIB y PIB Serie Desestacionalizada

In [64]:
try:
    driver.quit()
except:
    print("no driver open")       

In [65]:
driver  = start_driver()
time.sleep(10)
driver.get("https://www.indec.gob.ar/indec/web/Nivel3-Tema-3-9")
time.sleep(12)

In [66]:
# Define XPATHs
PIB = '/html/body/div[2]/div[1]/div[2]/div[3]/div/div[1]'
PIB_sd = '/html/body/div[2]/div[1]/div[2]/div[3]/div/div[2]'

box1 = f'{PIB}/div[1]'
box2 = f'{PIB_sd}/div[1]'

In [67]:
# Check for 'Producto interno bruto' in both boxes
try:
    box1_text = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, box1))
    ).text

    box2_text = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, box2))
    ).text

    if 'interno bruto' in box1_text and 'interno bruto' in box2_text:
        # Extracting KPI values and names
        data = {
            'PIB': {
                'Fecha' : today,
                'kpi_value': WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB}/div[2]'))
                ).text,
                'kpi_name': 'PIB - ' + WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB}/div[3]'))
                ).text,
                'kpi_period': WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB}/div[4]'))
                ).text,
            },
            'PIB - SD': {
                'Fecha' : today,
                'kpi_value': WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB_sd}/div[2]'))
                ).text,
                'kpi_name': 'PIB - ' + WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB_sd}/div[3]'))
                ).text + " - " + WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB_sd}/div[5]'))
                ).text,
                'kpi_period': WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'{PIB_sd}/div[4]'))
                ).text,
            }
        }

        # Print or process the data as needed
        print(data)

except Exception as e:
    print("An error occurred:", e)
time.sleep

{'PIB': {'Fecha': '2025-01-20', 'kpi_value': '-2,1%', 'kpi_name': 'PIB - Variación interanual', 'kpi_period': 'Tercer trimestre 2024'}, 'PIB - SD': {'Fecha': '2025-01-20', 'kpi_value': '3,9%', 'kpi_name': 'PIB - Variación trimestral - Serie desestacionalizada', 'kpi_period': 'Tercer trimestre 2024'}}


<function time.sleep>

In [68]:
df_4 = pd.DataFrame.from_dict(data, orient='index')
df_4.reset_index(inplace=True)
df_4.columns = ['KPI', 'Fecha', 'KPI Value', 'KPI Name', 'KPI Period']
df_4 = df_4[['Fecha', 'KPI Period', 'KPI Name', 'KPI Value']]
df_4

,Fecha,KPI Period,KPI Name,KPI Value
0,2025-01-20,Tercer trimestre 2024,PIB - Variación interanual,"-2,1%"
1,2025-01-20,Tercer trimestre 2024,PIB - Variación trimestral - Serie desestacion...,"3,9%"


### 4.2 Agregados Macroeconomicos


#### Descargamos el csv con la información actualizada e histórica del PIB desde la base de datos del gobierno.
#### Son dos archivos.


#### Archivo 1

In [69]:
try:
    driver.quit()
except:
    print("no driver open")       

In [70]:
driver = start_driver()

In [71]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/6/distribution/6.2/download/producto-interno-bruto-valores-trimestrales-base-2004.csv")
time.sleep(12)

In [72]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "producto-interno-bruto" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'producto-interno-bruto' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

Renamed 'producto-interno-bruto-valores-trimestrales-base-2004.csv' to '2025-01-20_producto-interno-bruto-valores-trimestrales-base-2004.csv'.


In [73]:
pib_1 = pd.read_csv(new_file_path)

In [74]:
pib_1

,indice_tiempo,agricultura_ganaderia_caza_silvicultura,pesca,explotacion_minas_canteras,industria_manufacturera,suministro_electricidad_gas_agua,construccion,sectores_productores_bienes,comercio_mayorista_minorista_reparaciones,hoteles_restaurantes,...,ensenianza,servicios_sociales_salud,otras_actividades_servicios_comunitarias_sociales_personales,hogares_privs_servicio_domestico,sectores_productores_servicios,valor_agregado_bruto_precios_basicos,impuesto_valor_agregado,derechos_importacion,impuestos_productos_netos_subsidios,producto_interno_bruto_precios_mercado
0,2004-01-01,31507.326987,1341.873285,24027.489534,86448.022169,8603.245520,14801.185460,166729.142954,58565.315974,7234.787298,...,16380.347905,12435.382071,11844.979806,2918.887328,224057.789755,390786.932709,29036.879929,2710.830711,37834.798884,460369.442233
1,2004-04-01,65745.371141,2140.640207,24441.463708,91700.169045,8742.807661,14555.301100,207325.752860,60894.070950,6353.045380,...,17003.853733,13606.637987,12086.240277,3062.277711,232416.832025,439742.584885,31112.298738,3170.234449,40370.563701,514395.681772
2,2004-07-01,28753.566747,1177.366832,24618.477685,93728.825751,8906.931528,15289.959686,172475.128229,60102.615771,6872.978025,...,17267.575984,13687.034254,12625.696734,3029.860090,234103.810984,406578.939213,33429.770668,3468.105636,37675.164427,481151.979944
3,2004-10-01,30494.525577,1110.102086,24199.792351,95588.304996,8556.637598,14972.455313,174921.817921,60576.379075,7283.326287,...,17577.818175,13076.359596,12457.239638,3160.369112,237679.562806,412601.380727,30328.847160,3652.642995,37960.805995,484543.676877
4,2005-01-01,35665.069110,1305.396247,23998.485412,91736.477998,9171.338180,16323.794888,178200.561835,63579.084865,7905.875450,...,17479.472146,13825.714354,12940.939060,3281.793623,241440.287765,419640.849600,31848.743728,3325.865472,38787.071778,493602.530578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2023-07-01,32018.250130,3824.403411,26203.391594,122769.178187,13194.067688,23679.258864,221688.549875,98376.894936,10342.057442,...,29534.898254,24520.070942,16132.657458,4071.863406,379885.007996,601573.557870,55724.209043,9210.238849,56682.586669,723190.592431
79,2023-10-01,35821.727033,1353.526596,26399.152545,116716.282150,12211.337351,21526.474957,214028.500632,91595.833336,11262.430451,...,29683.410501,23326.100403,15881.382358,4192.828104,371667.354920,585695.855552,51468.464839,7599.448814,51528.485644,696292.254849
80,2024-01-01,39511.979198,2069.555122,26607.599396,96336.630909,13467.942618,18512.891922,196506.599166,84181.034008,11186.163460,...,28871.571511,24553.045434,17240.099859,4359.858959,358719.777738,555226.376903,54103.878034,5976.395243,53366.063685,668672.713865
81,2024-04-01,83621.401897,2818.835344,27550.269747,101684.095532,12891.574302,17135.551034,245701.727856,85588.732088,9239.534016,...,29341.189488,26216.766606,14478.130527,3886.022546,362968.595333,608670.323188,51794.785654,5964.086060,54331.285204,720760.480106


In [75]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

Moved '2025-01-20_producto-interno-bruto-valores-trimestrales-base-2004.csv' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.


In [76]:
time.sleep(5)

#### Archivo 2

In [77]:
try:
    driver.quit()
except:
    print("no driver open")       

In [78]:
driver = start_driver()

In [79]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/9/distribution/9.2/download/producto-interno-bruto-precios-corrientes-valores-trimestrales-base-2004.csv")
time.sleep(12)

In [80]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "producto-interno-bruto" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'producto-interno-bruto' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

Renamed 'producto-interno-bruto-precios-corrientes-valores-trimestrales-base-2004.csv' to '2025-01-20_producto-interno-bruto-precios-corrientes-valores-trimestrales-base-2004.csv'.


In [81]:
pib_2 = pd.read_csv(new_file_path)

In [82]:
pib_2

,indice_tiempo,pib_precios_corrientes,pib_precios_2004,tcn_pesos_dolares,pib_dolares_precios_corrientes,poblacion,pib_per_capita_pesos_corrientes,pib_per_capita_dolares_corrientes,indice_precios_constantes
0,2004-01-01,4.391959e+05,460369.442233,2.906752,151095.081763,3.813141e+07,1.151795e+04,3962.483144,100.000000
1,2004-04-01,5.094987e+05,514395.681772,2.907355,175244.737312,3.823033e+07,1.332708e+04,4583.918820,111.446301
2,2004-07-01,4.884624e+05,481151.979944,2.988391,163453.324510,3.832951e+07,1.274377e+04,4264.425431,103.974168
3,2004-10-01,5.033039e+05,484543.676877,2.964778,169761.056929,3.842894e+07,1.309700e+04,4417.531654,104.436171
4,2005-01-01,5.107034e+05,493602.530578,2.929534,174329.244688,3.852863e+07,1.325517e+04,4524.667619,106.113398
...,...,...,...,...,...,...,...,...,...
78,2023-07-01,2.053537e+08,723190.592431,312.147560,657873.646605,4.665458e+07,4.401575e+06,14100.944269,128.391090
79,2023-10-01,2.821035e+08,696292.254849,446.752258,631453.946230,4.675750e+07,6.033330e+06,13504.868306,123.343608
80,2024-01-01,4.181918e+08,668672.713865,834.312456,501241.277080,4.686066e+07,8.924157e+06,10696.420431,118.190250
81,2024-04-01,5.550622e+08,720760.480106,885.913922,626541.861941,4.696403e+07,1.181888e+07,13340.886694,127.116515


In [83]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

Moved '2025-01-20_producto-interno-bruto-precios-corrientes-valores-trimestrales-base-2004.csv' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.


### 4.3 Indice de Salarios

In [45]:
try:
    driver.quit()
except:
    print("no driver open")       

In [46]:
driver = start_driver()

In [47]:
today

'2025-02-11'

In [48]:
def subtract_one_month(date_str):
    # Split the date string into year, month, and day
    year, month, day = map(int, date_str.split('-'))
    
    # Subtract 1 from the month
    if month == 1:
        month = 12
        year -= 1
    else:
        month -= 1
    
    # Format the new date as YY-MM
    new_date_str = f"{month:02d}_{year % 100:02d}"
    
    return new_date_str

# Given date


# Subtract 1 month and print the result
new_date_str = subtract_one_month(today)
print(new_date_str)

01_25


In [52]:
time.sleep(5)
driver.get(f"https://www.indec.gob.ar/ftp/cuadros/sociedad/variaciones_salarios_{new_date_str}.xls")
time.sleep(12)

In [53]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "salarios" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'salarios' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

Renamed 'variaciones_salarios_01_25.xls' to '2025-02-11_variaciones_salarios_01_25.xls'.


In [62]:
def extract_table2(file_path, sheet_name, start_col, end_col, start_row, end_row=None):
    # Construct the column range string
    col_range = f"{start_col}:{end_col}"
    
    # Read the specific columns and rows from the sheet
    if end_row == None:
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1)
    # Drop rows and columns that are completely empty
    else:
        nrows = end_row - start_row + 1
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1, nrows=nrows)
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    
    # Remove any leading or trailing whitespace from column names
    def is_date(string):
        try:
            pd.to_datetime(string)
            return True
        except ValueError:
            return False

# Verify if the column header is a date and format it as '2025-02-06' and convert it to text
    df.columns = [pd.to_datetime(col).strftime('%Y-%m-%d') if is_date(col) else col.strip() for col in df.columns]
    
    # Replace any remaining NaN values with an empty string
    df.fillna('', inplace=True)
    
    return df

#### Cuadro 1

In [92]:
sheet_name = 'Cuadro 1'
start_col = 'A'
end_col = 'L'
start_row = 0

# Extract the table
df_cuadro1 = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row)

In [93]:
months = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Filter the dataframe to only include rows where the second column contains any of the specified months
df_cuadro1.iloc[:, 1] = df_cuadro1.iloc[:, 1].str.strip()
filtered_df = df_cuadro1[df_cuadro1.iloc[:, 1].isin(months)]
filtered_df.iloc[:, 0].fillna(method='ffill', inplace=True)
# Display the filtered dataframe
filtered_df

,"Cuadro 1. Índice de salarios. Variaciones porcentuales respecto del período anterior y números índice, octubre 2016=100,0, por sector. Años 2015-2024",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
7,2015,Octubre,///,73.97,///,75.24,///,74.44,…,…,…,…
8,,Noviembre,2.68,75.95,1.37,76.28,2.19,76.07,…,…,…,…
9,,Diciembre,1.86,77.36,1.18,77.18,1.6,77.29,…,…,…,…
10,2016,Enero,2.02,78.93,0.36,77.46,1.41,78.38,…,…,…,…
11,,Febrero,1.41,80.04,1.16,78.36,1.32,79.42,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
112,,Julio,6.27,5450.78,6.72,4390.43,6.42,5054.69,15.08,3040.48,7.47,4653.26
113,,Agosto,5.04,5725.38,4.65,4594.73,4.91,5303.03,10.58,3362.05,5.65,4916.19
114,,Septiembre,3.77,5941.03,3.92,4774.83,3.82,5505.4,10.35,3710.18,4.71,5147.61
115,,Octubre,3.95,6175.77,4.34,4982.02,4.08,5729.85,7.66,3994.36,4.59,5383.96


In [94]:
def correct_blank_row(df):
    current_row = ""
    first_col = df.columns[0]
    #date_header =  df.columns[1]
    for index, row in df.iterrows():
        if row[first_col] != '':
            current_row = row[first_col]
        else:
            df.at[index, first_col] = current_row
    return df

In [95]:
cuadro1_variaciones_porcentuales_periodo_anterior = correct_blank_row(filtered_df)

In [96]:
columns_names = ["Año","Período",
"Sector_registrado_Sector_privado_registrado_Variación_porcentual_respecto_del_mes_anterior",
"Sector_registrado_Sector_privado_registrado_Número_índice",
"Sector_registrado_Sector_público_Variación_porcentual_respecto_del_mes_anterior",
"Sector_registrado_Sector_público_Número_índice",
"Sector_registrado_Total_registrado_(1)_Variación_porcentual_respecto_del_mes_anterior",
"Sector_registrado_Total_registrado_(1)_Número_índice",
"Sector_privado_no_registrado_(2)_Variación_porcentual_respecto_del_mes_anterior",
"Sector_privado_no_registrado_(2)_Número_índice",
"Total_índice_de_salarios_Variación_porcentual_respecto_del_mes_anterior",
"Total_índice_de_salarios_Número_índice"]
cuadro1_variaciones_porcentuales_periodo_anterior.columns = columns_names

In [97]:
cuadro1_variaciones_porcentuales_periodo_anterior

,Año,Período,Sector_registrado_Sector_privado_registrado_Variación_porcentual_respecto_del_mes_anterior,Sector_registrado_Sector_privado_registrado_Número_índice,Sector_registrado_Sector_público_Variación_porcentual_respecto_del_mes_anterior,Sector_registrado_Sector_público_Número_índice,Sector_registrado_Total_registrado_(1)_Variación_porcentual_respecto_del_mes_anterior,Sector_registrado_Total_registrado_(1)_Número_índice,Sector_privado_no_registrado_(2)_Variación_porcentual_respecto_del_mes_anterior,Sector_privado_no_registrado_(2)_Número_índice,Total_índice_de_salarios_Variación_porcentual_respecto_del_mes_anterior,Total_índice_de_salarios_Número_índice
7,2015,Octubre,///,73.97,///,75.24,///,74.44,…,…,…,…
8,2015,Noviembre,2.68,75.95,1.37,76.28,2.19,76.07,…,…,…,…
9,2015,Diciembre,1.86,77.36,1.18,77.18,1.6,77.29,…,…,…,…
10,2016,Enero,2.02,78.93,0.36,77.46,1.41,78.38,…,…,…,…
11,2016,Febrero,1.41,80.04,1.16,78.36,1.32,79.42,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
112,2024,Julio,6.27,5450.78,6.72,4390.43,6.42,5054.69,15.08,3040.48,7.47,4653.26
113,2024,Agosto,5.04,5725.38,4.65,4594.73,4.91,5303.03,10.58,3362.05,5.65,4916.19
114,2024,Septiembre,3.77,5941.03,3.92,4774.83,3.82,5505.4,10.35,3710.18,4.71,5147.61
115,2024,Octubre,3.95,6175.77,4.34,4982.02,4.08,5729.85,7.66,3994.36,4.59,5383.96


#### Cuadro 2

In [98]:
sheet_name = 'Cuadro 2'
start_col = 'A'
end_col = 'L'
start_row = 7

# Extract the table
df_cuadro2 = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row)

In [99]:
df_cuadro2

,Unnamed: 0,Unnamed: 1,igual mes del año anterior,diciembre del año anterior,igual mes del año anterior.1,diciembre del año anterior.1,igual mes del año anterior.2,diciembre del año anterior.2,igual mes del año anterior.3,diciembre del año anterior.3,igual mes del año anterior.4,diciembre del año anterior.4
0,,,%,,,,,,,,,
2,2016,Enero,///,2.02,///,0.36,///,1.41,…,…,…,…
3,,Febrero,///,3.47,///,1.53,///,2.74,…,…,…,…
4,,Marzo,///,4.44,///,8.58,///,5.99,…,…,…,…
5,,Abril,///,11.06,///,9.18,///,10.36,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
108,,Noviembre,167.37,140.79,127.42,115.57,152.99,132.01,198.87,177.68,159.12,138.04
110,(1) Se corresponde con lo establecido en el “A...,,,,,,,,,,,
111,(2) Ver “Acerca del relevamiento de la EPH des...,,,,,,,,,,,
112,https://www.indec.gob.ar/ftp/cuadros/menusuper...,,,,,,,,,,,


In [100]:
months = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Filter the dataframe to only include rows where the second column contains any of the specified months
df_cuadro2.iloc[:, 1] = df_cuadro2.iloc[:, 1].str.strip()
filtered_cuadro2 = df_cuadro2[df_cuadro2.iloc[:, 1].isin(months)]
filtered_cuadro2.iloc[:, 0].fillna(method='ffill', inplace=True)
# Display the filtered dataframe
filtered_cuadro2

,Unnamed: 0,Unnamed: 1,igual mes del año anterior,diciembre del año anterior,igual mes del año anterior.1,diciembre del año anterior.1,igual mes del año anterior.2,diciembre del año anterior.2,igual mes del año anterior.3,diciembre del año anterior.3,igual mes del año anterior.4,diciembre del año anterior.4
2,2016,Enero,///,2.02,///,0.36,///,1.41,…,…,…,…
3,,Febrero,///,3.47,///,1.53,///,2.74,…,…,…,…
4,,Marzo,///,4.44,///,8.58,///,5.99,…,…,…,…
5,,Abril,///,11.06,///,9.18,///,10.36,…,…,…,…
6,,Mayo,///,14.72,///,12.86,///,14.03,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
104,,Julio,235.06,105.78,169.96,84.94,210.74,98.52,178.86,95.44,206.19,98.11
105,,Agosto,224.85,116.14,166.63,93.54,203.41,108.27,184.1,116.12,200.62,109.31
106,,Septiembre,197.71,124.29,149.73,101.13,180.27,116.22,191.76,138.5,181.86,119.16
107,,Octubre,184.62,133.14,138.74,109.86,167.9,125.03,199.42,156.76,172.14,129.22


In [101]:
def correct_blank_row(df):
    current_row = ""
    first_col = df.columns[0]
    #date_header =  df.columns[1]
    for index, row in df.iterrows():
        if row[first_col] != '':
            current_row = row[first_col]
        else:
            df.at[index, first_col] = current_row
    return df

In [105]:
cuadro2_variaciones_porcentuales_acumuladas = correct_blank_row(filtered_cuadro2)

In [106]:
columns_names = ["Año",
 "Período",
"Sector_registrado_Sector_privado_registrado_igual_mes_del_año_anterior",
"Sector_registrado_Sector_privado_registrado_diciembre_del_año_anterior",
"Sector_registrado_Sector_público_igual_mes_del_año_anterior",
"Sector_registrado_Sector_público_diciembre_del_año_anterior",
"Sector_registrado_Total_registrado_(1)_igual_mes_del_año_anterior",
"Sector_registrado_Total_registrado_(1)_diciembre_del_año_anterior",
"Sector_privado_no_registrado_(2)_igual_mes_del_año_anterior",
"Sector_privado_no_registrado_(2)_diciembre_del_año_anterior",
"Total_índice_de_salarios_igual_mes_del_año_anterior",
"Total_índice_de_salarios_diciembre_del_año_anterior"]
cuadro2_variaciones_porcentuales_acumuladas.columns = columns_names

In [107]:
cuadro2_variaciones_porcentuales_acumuladas

,Año,Período,Sector_registrado_Sector_privado_registrado_igual_mes_del_año_anterior,Sector_registrado_Sector_privado_registrado_diciembre_del_año_anterior,Sector_registrado_Sector_público_igual_mes_del_año_anterior,Sector_registrado_Sector_público_diciembre_del_año_anterior,Sector_registrado_Total_registrado_(1)_igual_mes_del_año_anterior,Sector_registrado_Total_registrado_(1)_diciembre_del_año_anterior,Sector_privado_no_registrado_(2)_igual_mes_del_año_anterior,Sector_privado_no_registrado_(2)_diciembre_del_año_anterior,Total_índice_de_salarios_igual_mes_del_año_anterior,Total_índice_de_salarios_diciembre_del_año_anterior
2,2016,Enero,///,2.02,///,0.36,///,1.41,…,…,…,…
3,2016,Febrero,///,3.47,///,1.53,///,2.74,…,…,…,…
4,2016,Marzo,///,4.44,///,8.58,///,5.99,…,…,…,…
5,2016,Abril,///,11.06,///,9.18,///,10.36,…,…,…,…
6,2016,Mayo,///,14.72,///,12.86,///,14.03,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
104,2024,Julio,235.06,105.78,169.96,84.94,210.74,98.52,178.86,95.44,206.19,98.11
105,2024,Agosto,224.85,116.14,166.63,93.54,203.41,108.27,184.1,116.12,200.62,109.31
106,2024,Septiembre,197.71,124.29,149.73,101.13,180.27,116.22,191.76,138.5,181.86,119.16
107,2024,Octubre,184.62,133.14,138.74,109.86,167.9,125.03,199.42,156.76,172.14,129.22


In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

## 7.Informe monetario mensual - Pendiente definición.

In [93]:
try:
    driver.quit()
except:
    print("no driver open")       

In [94]:
driver = start_driver()

In [95]:
time.sleep(5)
driver.get("https://www.bcra.gob.ar/PublicacionesEstadisticas/Informe_monetario_mensual.asp")
time.sleep(5)

In [188]:
element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Informe Monetario Diario.xls')]"))
    )
element.click()

In [189]:
file_to_rename = None
for filename in os.listdir(source_directory):
    if "infomondia" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'producto-interno-bruto' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")


Renamed 'infomondia.xls' to '2025-01-20_infomondia.xls'.


In [196]:
infomondia = pd.read_excel(new_file_path)


In [199]:
caratula_datos = pd.read_excel(new_file_path, sheet_name='DATOS | DATA')

cuadro_1 = pd.read_excel(new_file_path, sheet_name='CUADRO 1 | TABLE 1')


cuadro_2 = pd.read_excel(new_file_path, sheet_name='CUADRO 2 | TABLE 2')



### ADEFA

In [110]:
if today.split("-")[1] == "01":
    year = int(today.split("-")[0]) - 1
    month = "12"
    print(year, month)
else:
    year = today.split("-")[0]
    month = int(today.split("-")[1])-1
    print(year,month)

2025 1


In [111]:
url_adefa = f"https://adefa.org.ar/es/estadisticas-mensuales?ida={year}"
search_string = f"resumen-{year}-{month:02}"

In [112]:
try:
    driver.quit()
except:
    print("no driver open")   

In [113]:
driver = start_driver()
time.sleep(5)
driver.get(url_adefa)
time.sleep(12)

In [114]:
target_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//input[contains(@onclick, '{search_string}')]"))
    )


target_input.click()


In [115]:
pdf_url = driver.current_url
response = requests.get(pdf_url, verify=False)

C:\Users\bd1266\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adefa.org.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [116]:
response


<Response [200]>

In [117]:
pdf_url = driver.current_url

In [118]:
response = requests.get(pdf_url, verify=False)
with open('document.pdf', 'wb') as f:
    f.write(response.content)

C:\Users\bd1266\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adefa.org.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [119]:
with open('document.pdf', 'rb') as f:
    pdf_reader = PdfReader(f)
    if len(pdf_reader.pages) >= 2: 
        page_prod2 = pdf_reader.pages[1]  
        
        text_page_prod2 = page_prod2.extract_text()  
        print("Text from page 3:", text_page_prod2)
    else:
        print("The PDF does not have enough pages.")

Text from page 3: La producción nacional de Enero registró una baja del 20,9% respecto de 
Diciembre y un ascenso del 32,7% con relación a igual mes del año pasado.Producción
 30.058 u.
 
Las exportaciones de Enero tuvieron una baja del 58,3% con relación a 
Diciembre y un descenso del 27,3% con relación a igual mes del año anterior. Exportaciones
 11.132 u.
 
Las ventas a concesionarios de Enero tuvieron una baja del 31,1% respecto de 
Diciembre y un ascenso del 113,8% con relación a igual mes del año pasado.Ventas 
Mayoristas
34.089 u.
 Resumen enero 2025012025 INFORME DE PRENSA



In [120]:
year_month = str(year) + "-" + str(month)
df_adefa = pd.DataFrame(columns=["Año-Mes", "Indicador", "Valor"])

# Use regex to find matches
match_produccion = re.search(r'Producción\s+(\d{1,3}(?:\.\d{3})*)\s*u', text_page_prod2)
match_exportaciones = re.search(r'Exportaciones\s+(\d{1,3}(?:\.\d{3})*)\s*u', text_page_prod2)                        
match_ventasmay = re.search(r'Mayoristas\s+(\d{1,3}(?:\.\d{3})*)\s*u', text_page_prod2)                        

# Check for production match
if match_produccion:
    production_number = match_produccion.group(1)  # Extract the matched number
    print("Production:", production_number)
    df_production = pd.DataFrame({"Año-Mes": year_month, "Indicador": ["Producción"], "Valor": [production_number]})
    df_adefa = pd.concat([df_adefa, df_production], ignore_index=True)
else:
    print("No match found for production.")

# Check for export match
if match_exportaciones:
    export_number = match_exportaciones.group(1)  # Extract the matched number
    print("Exportación:", export_number)
    # Create a DataFrame for the new row
    df_export = pd.DataFrame({"Año-Mes": year_month, "Indicador": ["Exportación"], "Valor": [export_number]})
    df_adefa = pd.concat([df_adefa, df_export], ignore_index=True)
else:
    print("No match found for export.")

# Check for ventas mayoristas match
if match_ventasmay:
    ventmay_number = match_ventasmay.group(1)  # Extract the matched number
    print("Ventas Mayoristas:", ventmay_number)
    df_ventamay = pd.DataFrame({"Año-Mes": year_month, "Indicador": ["Ventas Mayoristas"], "Valor": [ventmay_number]})
    df_adefa = pd.concat([df_adefa, df_ventamay], ignore_index=True)
else:
    print("No match found for ventas mayoristas.")

# Display the final DataFrame
df_adefa

Production: 30.058
Exportación: 11.132
Ventas Mayoristas: 34.089


,Año-Mes,Indicador,Valor
0,2025-1,Producción,30.058
1,2025-1,Exportación,11.132
2,2025-1,Ventas Mayoristas,34.089


### AFIP.GOB

In [192]:
try:
    driver.quit()
except:
    print("no driver open")   

In [193]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.afip.gob.ar/institucional/")
time.sleep(12)

In [194]:
driver.execute_script("window.scrollBy(0, 600);")

link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Ver recaudación de')]"))
    )


In [199]:
try:
    link.click()
except Exception as e:
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 500);")
    try:
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Ver recaudación de')]"))
        )
        link.click()
    except Exception as e:
        print("Couldn't do it")

In [196]:
driver.window_handles

['89E2AF174F2D1A8B0BBF637A15941C94']

In [200]:
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)




In [201]:
driver.execute_script("window.scrollBy(0, 1500);")

In [202]:
# Locate the download button using the provided XPath and click it
download_button = driver.find_element(By.XPATH, '/html/body/pdf-viewer//viewer-toolbar//div/div[3]/viewer-download-controls//cr-icon-button//div/cr-icon//svg')
download_button.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/pdf-viewer//viewer-toolbar//div/div[3]/viewer-download-controls//cr-icon-button//div/cr-icon//svg"}
  (Session info: chrome=132.0.6834.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66FA102F5+28725]
	(No symbol) [0x00007FF66F972AE0]
	(No symbol) [0x00007FF66F80510A]
	(No symbol) [0x00007FF66F8593D2]
	(No symbol) [0x00007FF66F8595FC]
	(No symbol) [0x00007FF66F8A3407]
	(No symbol) [0x00007FF66F87FFEF]
	(No symbol) [0x00007FF66F8A0181]
	(No symbol) [0x00007FF66F87FD53]
	(No symbol) [0x00007FF66F84A0E3]
	(No symbol) [0x00007FF66F84B471]
	GetHandleVerifier [0x00007FF66FD3F30D+3366989]
	GetHandleVerifier [0x00007FF66FD512F0+3440688]
	GetHandleVerifier [0x00007FF66FD478FD+3401277]
	GetHandleVerifier [0x00007FF66FADAAAB+858091]
	(No sym

In [203]:
with open('document.pdf', 'rb') as f:
    pdfplumber.open(f)
    page = pdf.pages[2]
    text = page.extract_text()

    print("Extracted text from page 3:")
    #print(text)
    
    

AttributeError: module 'pdfplumber' has no attribute 'pages'

AttributeError: '_io.BufferedReader' object has no attribute 'pages'

In [135]:
if iva_row:
    numbers = iva_row.split()[1:5]
    print("Numbers extracted from the IVA row:")
    print(numbers)
else:
    print("No row starting with 'IVA' found.")

NameError: name 'iva_row' is not defined

In [134]:
iva_df = pd.DataFrame(columns=["Mes", "Recaudación", "Variación Interanual", "Ene-Dic", "Dif Ene-Dic vs LY"])
row = {"Mes":year_month,
       "Recaudación":numbers[0],
       "Variación Interanual":numbers[1], 
       "Ene-Dic":numbers[2], 
       "Dif Ene-Dic vs LY":numbers[3] 
      }
iva_df = pd.concat([iva_df, pd.DataFrame([row])], ignore_index=True)

# Display the updated DataFrame
iva_df


NameError: name 'numbers' is not defined

### DNRPA

#### Autos

In [204]:
try:
    driver.quit()
except:
    print("no driver open")   

In [205]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones")
time.sleep(3)
driver.execute_script("window.scrollBy(0, 600);")
time.sleep(3)

In [206]:
current_year = today.split("-")[0]

##### BORRAR ESTA LINEA LUEGO. E PARA NO DESCARGAR LA TABLA DE 2025 VACÍA.

In [207]:
current_year = str(year) #BORRAR ESTA LINEA LUEGO. E PARA NO DESCARGAR LA TABLA DE 2025 VACÍA.

In [208]:
current_year

'2025'

In [209]:
dropdown = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select"))
)

# Create a Select object
select = Select(dropdown)

# Select the option with value "2024"
select.select_by_value(current_year)
time.sleep(2)

In [210]:
button_auto = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/center/table/tbody/tr[4]/td/input[1]"))
)

# Click the button
button_auto.click()
time.sleep(2)

In [211]:
button_aceptar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/center/center/input"))
)

# Click the button
button_aceptar.click()
time.sleep(2)

In [212]:

# Switch to the new window
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)


driver.switch_to.window(driver.window_handles[1])


In [213]:
driver.execute_script("window.scrollBy(0, 700);")

In [214]:

# Wait for the first row to be present
first_row = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/div/table/tbody/tr[1]"))
)

# Extract the column names from the first row
columns = [col.text for col in first_row.find_elements(By.XPATH, ".//th")]

# Initialize a list to store the table data
table_data = []

# Loop through each row from 3 to 26
for i in range(3, 27):
    # Construct the XPath for the current row
    row_xpath = f"/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/div/table/tbody/tr[{i}]"
    
    # Wait for the row to be present
    row = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, row_xpath))
    )
    
    # Extract the first element text inside the row
    first_element_xpath = f"{row_xpath}/td[1]/a"
    first_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, first_element_xpath))
    )
    first_element_text = first_element.text
    
    # Extract the rest of the columns in the row
    cols = [first_element_text] + [col.text for col in row.find_elements(By.XPATH, ".//td[position()>1]")]
    
    # Append the row data to the table_data list
    table_data.append(cols)

# Create a DataFrame from the table data
df_auto = pd.DataFrame(table_data, columns=columns)


In [215]:
df_auto

,Provincia / Mes,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Total
0,BUENOS AIRES,19.281,0,0,0,0,0,0,0,0,0,0,0,19.281
1,C.AUTONOMA DE BS.AS,12.251,0,0,0,0,0,0,0,0,0,0,0,12.251
2,CATAMARCA,569,0,0,0,0,0,0,0,0,0,0,0,569
3,CORDOBA,7.894,0,0,0,0,0,0,0,0,0,0,0,7.894
4,CORRIENTES,1.554,0,0,0,0,0,0,0,0,0,0,0,1.554
5,CHACO,1.604,0,0,0,0,0,0,0,0,0,0,0,1.604
6,CHUBUT,1.306,0,0,0,0,0,0,0,0,0,0,0,1.306
7,ENTRE RIOS,1.725,0,0,0,0,0,0,0,0,0,0,0,1.725
8,FORMOSA,471,0,0,0,0,0,0,0,0,0,0,0,471
9,JUJUY,819,0,0,0,0,0,0,0,0,0,0,0,819


In [216]:
driver.quit()

#### Motos

In [217]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones")
time.sleep(3)
driver.execute_script("window.scrollBy(0, 600);")
time.sleep(3)

In [218]:
dropdown = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select"))
)

# Create a Select object
select = Select(dropdown)

# Select the option with value "2024"
select.select_by_value(current_year)
time.sleep(2)

In [219]:
button_moto = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/center/table/tbody/tr[4]/td/input[2]"))
)

# Click the button
button_moto.click()
time.sleep(2)

In [220]:
button_aceptar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/center/center/input"))
)

# Click the button
button_aceptar.click()
time.sleep(2)

In [221]:

# Switch to the new window
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)


driver.switch_to.window(driver.window_handles[1])


In [222]:
driver.execute_script("window.scrollBy(0, 700);")

In [223]:

# Wait for the first row to be present
first_row = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/div/table/tbody/tr[1]"))
)

# Extract the column names from the first row
columns = [col.text for col in first_row.find_elements(By.XPATH, ".//th")]

# Initialize a list to store the table data
table_data = []

# Loop through each row from 3 to 26
for i in range(3, 27):
    # Construct the XPath for the current row
    row_xpath = f"/html/body/div[2]/section/article/div/div[2]/div/div/div/div/form/div/div/table/tbody/tr[{i}]"
    
    # Wait for the row to be present
    row = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, row_xpath))
    )
    
    # Extract the first element text inside the row
    first_element_xpath = f"{row_xpath}/td[1]/a"
    first_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, first_element_xpath))
    )
    first_element_text = first_element.text
    
    # Extract the rest of the columns in the row
    cols = [first_element_text] + [col.text for col in row.find_elements(By.XPATH, ".//td[position()>1]")]
    
    # Append the row data to the table_data list
    table_data.append(cols)

# Create a DataFrame from the table data
df_moto = pd.DataFrame(table_data, columns=columns)


In [224]:
df_moto

,Provincia / Mes,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Total
0,BUENOS AIRES,19.769,0,0,0,0,0,0,0,0,0,0,0,19.769
1,C.AUTONOMA DE BS.AS,2.056,0,0,0,0,0,0,0,0,0,0,0,2.056
2,CATAMARCA,1.144,0,0,0,0,0,0,0,0,0,0,0,1.144
3,CORDOBA,6.390,0,0,0,0,0,0,0,0,0,0,0,6.390
4,CORRIENTES,1.954,0,0,0,0,0,0,0,0,0,0,0,1.954
5,CHACO,3.119,0,0,0,0,0,0,0,0,0,0,0,3.119
6,CHUBUT,586,0,0,0,0,0,0,0,0,0,0,0,586
7,ENTRE RIOS,1.640,0,0,0,0,0,0,0,0,0,0,0,1.640
8,FORMOSA,828,0,0,0,0,0,0,0,0,0,0,0,828
9,JUJUY,444,0,0,0,0,0,0,0,0,0,0,0,444


### GRUPO CONSTRUYA

In [225]:
try:
    driver.quit()
except:
    print("no driver open")   

In [226]:
driver = start_driver()
time.sleep(5)
driver.get("https://www.grupoconstruya.com.ar/servicios/indice_construya")
time.sleep(3)
driver.execute_script("window.scrollBy(0, 1500);")
time.sleep(3)

In [227]:
table_element = driver.find_element(By.XPATH, '/html/body/section/div[1]/article/div[1]/div/div[2]')

# Find all rows in the table
rows = table_element.find_elements(By.CLASS_NAME, 'TablaRowIndice')

# Extract data from each row
table_data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, 'div')
    cell_data = [cell.get_attribute('innerHTML').split('</span>')[-1].strip() for cell in cells]
    table_data.append(cell_data)


# Create a DataFrame from the table data
df_indice_construya = pd.DataFrame(table_data, columns=[
    "Mes", "Indice Con Estacionalidad", "Variación % Interanual", 
    "Variación % Anual Acumulada", "Indice Desestacionalizado", "Variación % Mensual"
])
if df_indice_construya["Mes"].iloc[0] == "Mes":
    df_indice_construya = df_indice_construya.drop(0).reset_index(drop=True)

# Print the DataFrame
df_indice_construya

,Mes,Indice Con Estacionalidad,Variación % Interanual,Variación % Anual Acumulada,Indice Desestacionalizado,Variación % Mensual
0,<strong>Enero 2025</strong>,"247,2","-0,7%",N/A,"223,1","-7,8%"
1,Diciembre 2024,"244,2","-6,6%","-27,2%","242,0","-1,3%"
2,Noviembre 2024,"275,9","-24,8%","-28,5%","245,0","-3,2%"
3,Octubre 2024,"296,5","-28,2%","-28,7%","253,2","-2,8%"
4,Septiembre 2024,"293,7","-22,0%","-28,6%","260,5","-5,4%"
...,...,...,...,...,...,...
92,Mayo 2017,"340,1","14,5%","+3,5%","321,5","+3,3%"
93,Abril 2017,"305,6","6,6%","+0,9%","311,2","+1,3%"
94,Marzo 2017,"343,9","11,2%","-0,9%","307,4","+3,3%"
95,Febrero 2017,"281,0","-6,7%","-6,8%","297,7","+8,2%"


In [228]:
last_month_construya = df_indice_construya.iloc[0]
last_month_construya

Mes                            <strong>Enero 2025</strong>
Indice Con Estacionalidad                            247,2
Variación % Interanual                               -0,7%
Variación % Anual Acumulada                            N/A
Indice Desestacionalizado                            223,1
Variación % Mensual                                  -7,8%
Name: 0, dtype: object

### CAMMESA

In [229]:
try:
    driver.quit()
except:
    print("no driver open")       

In [230]:
driver = start_driver()

In [231]:
time.sleep(5)
driver.get("https://cammesaweb.cammesa.com/estadistica-informe-sintesis-mem/")
time.sleep(15)

In [232]:
button_demanda = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/article/div/div/div/div/div/div/div[3]/h3"))
)

# Click the button
button_demanda.click()
time.sleep(2)

In [233]:
button_descarga_demanda = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/article/div/div/div/div/div/div/div[3]/div/table/tbody/tr[1]/td[2]/div/div/div/div/div[3]/a"))
)

# Click the button
button_descarga_demanda.click()
time.sleep(2)


In [234]:
button_oferta = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/article/div/div/div/div/div/div/div[4]/h3"))
)

# Click the button
button_oferta.click()
time.sleep(2)

In [235]:
button_descarga__generacion_local = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/article/div/div/div/div/div/div/div[4]/div/table/tbody/tr[2]/td[2]/div/div/div/div/div[3]/a"))
)

# Click the button
button_descarga__generacion_local.click()
time.sleep(2)

In [236]:
button_descarga__combustible = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/article/div/div/div/div/div/div/div[4]/div/table/tbody/tr[3]/td[2]/div/div/div/div/div[3]/a"))
)

# Click the button
button_descarga__combustible.click()
time.sleep(2)

In [237]:
files = os.listdir(source_directory)

# Initialize variables to store DataFrames
combustibles_mensual_df = None
demanda_mensual_df = None
generacion_local_mensual_df = None

for file in files:
    file_path = os.path.join(source_directory, file)
    
    # Check if the file name contains "Combustibles"
    if "Combustibles" in file and file.endswith('.xlsx'):
        combustibles_mensual_df = pd.read_excel(file_path, engine='openpyxl')
        new_filename = f"{today}_{file}"
        new_file_path = os.path.join(source_directory, new_filename)
        os.rename(file_path, new_file_path)
        try:
            shutil.move(new_file_path, backup_directory)
            print(f"Moved '{new_filename}' to '{backup_directory}'.")
        except Exception as e:
            print(f"Error moving the file: {e}")
    
    # Check if the file name contains "Demanda"
    elif "Demanda" in file and file.endswith('.xlsx'):
        demanda_mensual_df = pd.read_excel(file_path, engine='openpyxl')
        new_filename = f"{today}_{file}"
        new_file_path = os.path.join(source_directory, new_filename)
        os.rename(file_path, new_file_path)
        try:
            shutil.move(new_file_path, backup_directory)
            print(f"Moved '{new_filename}' to '{backup_directory}'.")
        except Exception as e:
            print(f"Error moving the file: {e}")
    
    # Check if the file name contains "Generación"
    elif "Generación" in file and file.endswith('.xlsx'):
        generacion_local_mensual_df = pd.read_excel(file_path, engine='openpyxl')
        new_filename = f"{today}_{file}"
        new_file_path = os.path.join(source_directory, new_filename)
        os.rename(file_path, new_file_path)
        try:
            shutil.move(new_file_path, backup_directory)
            print(f"Moved '{new_filename}' to '{backup_directory}'.")
        except Exception as e:
            print(f"Error moving the file: {e}")

# Print the shapes of the DataFrames to confirm they were read correctly
if combustibles_mensual_df is not None:
    print(f"Combustibles Mensual DataFrame shape: {combustibles_mensual_df.shape}")
else:
    print("Combustibles Mensual DataFrame not found.")

if demanda_mensual_df is not None:
    print(f"Demanda Mensual DataFrame shape: {demanda_mensual_df.shape}")
else:
    print("Demanda Mensual DataFrame not found.")

if generacion_local_mensual_df is not None:
    print(f"Generación Local Mensual DataFrame shape: {generacion_local_mensual_df.shape}")
else:
    print("Generación Local Mensual DataFrame not found.")

Moved '2025-02-11_Combustibles Mensual.xlsx' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.
Moved '2025-02-11_Demanda Mensual.xlsx' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.
Moved '2025-02-11_Generación Local Mensual.xlsx' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.
Combustibles Mensual DataFrame shape: (17893, 49)
Demanda Mensual DataFrame shape: (50166, 49)
Generación Local Mensual DataFrame shape: (29628, 49)


##### Limpiamos los 3 excels para quedarnos con la primer tabla

In [238]:

header_row_index = demanda_mensual_df.index[demanda_mensual_df.iloc[:, 0] == 'TIPO'][0]

# Find the end of the table by looking for the "DEMANDA LOCAL" row
end_row_index = demanda_mensual_df.index[demanda_mensual_df.iloc[:, 0] == 'DEMANDA LOCAL'][0]

# Extract the table data from the DataFrame, including the "DEMANDA LOCAL" row
demanda_mensual_df_primer_tabla = demanda_mensual_df.iloc[header_row_index:end_row_index + 1].reset_index(drop=True)

# Set the first row as the header
demanda_mensual_df_primer_tabla.columns = demanda_mensual_df_primer_tabla.iloc[0]
demanda_mensual_df_primer_tabla = demanda_mensual_df_primer_tabla[1:]

In [239]:
# Assuming table_df is already loaded

# Convert the column headers to datetime, ignoring non-date columns
date_columns = pd.to_datetime(demanda_mensual_df_primer_tabla.columns[1:], errors='coerce')

# Find the max date
max_date = date_columns.max()

# Get the column name corresponding to the max date
max_date_column = demanda_mensual_df_primer_tabla.columns[1:][date_columns == max_date][0]

# Filter the DataFrame by only that column
demanda_mensual_final = demanda_mensual_df_primer_tabla[['TIPO', max_date_column]]

# Print the filtered DataFrame
demanda_mensual_final

,TIPO,2024-12-01 00:00:00
1,Demanda Residencial,5164217.448
2,Demanda Comercial,3379395.467
3,Gran Usuario en Distribuidor,1083985.866
4,Gran Usuario MEM,1877835.788
5,Gran Demanda Industrial/Comercial,2961821.654
6,DEMANDA LOCAL,11505434.569


In [240]:

header_row_index = generacion_local_mensual_df.index[generacion_local_mensual_df.iloc[:, 0] == 'TIPO'][0]

# Find the end of the table by looking for the "DEMANDA LOCAL" row
end_row_index = generacion_local_mensual_df.index[generacion_local_mensual_df.iloc[:, 0] == 'TOTAL [MWh]'][0]

# Extract the table data from the DataFrame, including the "DEMANDA LOCAL" row
generacion_df_primer_tabla = generacion_local_mensual_df.iloc[header_row_index:end_row_index + 1].reset_index(drop=True)

# Set the first row as the header
generacion_df_primer_tabla.columns = generacion_df_primer_tabla.iloc[0]
generacion_df_primer_tabla = generacion_df_primer_tabla[1:]

In [241]:
# Assuming table_df is already loaded

# Convert the column headers to datetime, ignoring non-date columns
date_columns = pd.to_datetime(generacion_df_primer_tabla.columns[1:], errors='coerce')

# Find the max date
max_date = date_columns.max()

# Get the column name corresponding to the max date
max_date_column = generacion_df_primer_tabla.columns[1:][date_columns == max_date][0]

# Filter the DataFrame by only that column
generacion_mensual_final = generacion_df_primer_tabla[['TIPO', max_date_column]]

# Print the filtered DataFrame
generacion_mensual_final

,TIPO,2024-12-01 00:00:00
1,Térmica,5828642.602
2,Nuclear,676135.787
3,Renovable (incluye Hidro),5436168.552
4,Renovable,2195935.47
5,Hidráulica,3240233.082
6,TOTAL [MWh],11940946.941


In [242]:

header_row_index = combustibles_mensual_df.index[combustibles_mensual_df.iloc[:, 0] == 'TIPO'][0]

# Find the end of the table by looking for the "DEMANDA LOCAL" row
end_row_index = combustibles_mensual_df.index[combustibles_mensual_df.iloc[:, 0] == 'BIODIESEL'][0]

# Extract the table data from the DataFrame, including the "DEMANDA LOCAL" row
combustibles_mensual_df_primer_tabla = combustibles_mensual_df.iloc[header_row_index:end_row_index + 1].reset_index(drop=True)

# Set the first row as the header
combustibles_mensual_df_primer_tabla.columns = combustibles_mensual_df_primer_tabla.iloc[0]
combustibles_mensual_df_primer_tabla = combustibles_mensual_df_primer_tabla[1:]

In [243]:
# Assuming table_df is already loaded

# Convert the column headers to datetime, ignoring non-date columns
date_columns = pd.to_datetime(combustibles_mensual_df_primer_tabla.columns[1:], errors='coerce')

# Find the max date
max_date = date_columns.max()

# Get the column name corresponding to the max date
max_date_column = combustibles_mensual_df_primer_tabla.columns[1:][date_columns == max_date][0]

# Filter the DataFrame by only that column
combustible_mensual_final = combustibles_mensual_df_primer_tabla[['TIPO', max_date_column]]

# Print the filtered DataFrame
combustible_mensual_final

,TIPO,2024-12-01 00:00:00
1,GAS NATURAL,1255366.134
2,FUEL OIL,10.513
3,GAS OIL,4887.279
4,CARBÓN MINERAL,0
5,BIODIESEL,0


# DATOS FINANCIEROS DEL MERCADO LOCAL


## 1.1 Rava

### 1.1 Dolar

In [244]:
try:
    driver.quit()
except:
    print("no driver open")       

In [245]:
driver = start_driver()

In [246]:
time.sleep(5)
driver.get("https://www.rava.com/")
time.sleep(15)

In [247]:


body_xpath= '/html/body/div[1]/main/div/div/div[2]/div[1]/div[2]/table/tbody'


list_of_names = ["MEP", "CCL", "MAYORISTA"]

In [248]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)
# Count the number of child elements
item_count = len(child_elements)
print(item_count)
results =[]

list_of_names

df_8 = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Assuming child_elements is already defined and contains the relevant elements
for i in child_elements:
    split_values = i.text.split()

    # Ensure there are at least 3 parts before adding to DataFrame
    if len(split_values) >= 3:
        # Check if the first part is in the list_of_names
        if split_values[0] in list_of_names:
            # Check if the second part is not "LEDE" or "SENEBI"
            if split_values[1] not in ["LEDE", "SENEBI"]:
                # Create a new row as a DataFrame
                new_row = pd.DataFrame({
                    "Fecha" : today,
                    "Name": [split_values[0]],
                    "Valor": [split_values[1]],
                    "% dia": [split_values[2]]
                })
                # Append the new row to the existing DataFrame using concat
                df_8 = pd.concat([df_8, new_row], ignore_index=True)
            else:
                print(f"Skipping entry with 'LEDE' or 'SENEBI': {split_values}")
        else:
            print(f"Value not useful: {split_values}")
    else:
        print(f"Not enough parts in split_values: {split_values}")

# Display the final DataFrame


10
Value not useful: ['OFICIAL', '1.076,00', '0,1', '0,2', '2,2']
Value not useful: ['TARJETA', '1.398,80', '0,1', '0,2', '2,2']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'LEDE', '1.174,37', '0,1', '0,5', '1,3']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'SENEBI', '1.162,75', '-1,6', '-0,3', '-0,3']
Value not useful: ['OFI-MEP', '9,70', '-', '14,1', '-13,4']
Value not useful: ['OFI-CCL', '10,20', '-', '-1,9', '-20,3']
Value not useful: ['MEP-CCL', '0,40', '-', '-77,8', '-71,4']


In [249]:
df_8

,Fecha,Name,Valor,% dia
0,2025-02-11,MAYORISTA,"1.055,25",-
1,2025-02-11,MEP,"1.180,85","-0,3"
2,2025-02-11,CCL,"1.185,28","0,2"


### 1.2 Bonos

In [250]:
try:
    driver.quit()
except:
    print("no driver open")       

In [251]:
driver = start_driver()

In [252]:
time.sleep(5)
driver.get("https://www.rava.com/cotizaciones/bonos")
time.sleep(15)

In [253]:


body_xpath= '/html/body/div[1]/main/div/div/div[2]/table/tbody'


list_of_names = ["AL30", "AL30D", "AL30C", "GD30", "GD30D", "GD30C" , "BPY26"]

In [254]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)
# Count the number of child elements
item_count = len(child_elements)
print(item_count)
results =[]

list_of_names

df_bonos = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Assuming child_elements is already defined and contains the relevant elements
for i in child_elements:
    split_values = i.text.split()

    # Ensure there are at least 3 parts before adding to DataFrame
    if len(split_values) >= 3:
        # Check if the first part is in the list_of_names
        if split_values[0] in list_of_names:
            
                new_row = pd.DataFrame({
                    "Fecha" : today,
                    "Name": [split_values[0]],
                    "Valor": [split_values[1]],
                    "% dia": [split_values[2]]
                })
                # Append the new row to the existing DataFrame using concat
                df_bonos = pd.concat([df_bonos, new_row], ignore_index=True)
        
        else:
            pass
    else:
        print(f"Not enough parts in split_values: {split_values}")

# Display the final DataFrame


64


In [255]:
df_bonos

,Fecha,Name,Valor,% dia
0,2025-02-11,AL30,"76.330,00","-0,29"
1,2025-02-11,AL30C,"64,08","-0,26"
2,2025-02-11,AL30D,"64,64","-0,02"
3,2025-02-11,BPY26,"113.400,00","-0,53"
4,2025-02-11,GD30,"77.290,00","-0,34"
5,2025-02-11,GD30C,"64,75","-0,38"
6,2025-02-11,GD30D,"65,31","-0,21"


### 1.3 Lo mejor - Lo peor

In [256]:
try:
    driver.quit()
except:
    print("no driver open")       

In [257]:
driver = start_driver()

In [258]:
time.sleep(5)
driver.get("https://www.rava.com")
time.sleep(15)

In [259]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[8]/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_lomejor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Name": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_lomejor = pd.concat([df_lomejor, new_row], ignore_index=True)

print("Lo mejor")
df_lomejor

5
Lo mejor


,Fecha,Name,Valor,% dia
0,2025-02-11,DOME,"74,50",-
1,2025-02-11,LEDE,"1.435,00",-
2,2025-02-11,GARO,"350,00","-0,4"
3,2025-02-11,AUSO,"3.245,00","-0,6"
4,2025-02-11,VALO,"382,00","-0,6"


In [260]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[9]/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_lopeor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Name": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_lopeor = pd.concat([df_lopeor, new_row], ignore_index=True)

print("Lo peor")
df_lopeor

5
Lo peor


,Fecha,Name,Valor,% dia
0,2025-02-11,RIGO,"826,00","-10,2"
1,2025-02-11,BPAT,"2.260,00","-7,8"
2,2025-02-11,SEMI,"28,20","-7,4"
3,2025-02-11,SUPV,"3.350,00","-7,1"
4,2025-02-11,TECO2,"2.580,00","-6,9"


### 1.4 Acciones Merval

In [261]:
try:
    driver.quit()
except:
    print("no driver open")       

In [262]:
driver = start_driver()

In [263]:
time.sleep(5)
driver.get("https://www.rava.com/cotizaciones/acciones-argentinas")
time.sleep(15)

In [264]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_merval = pd.DataFrame(columns=["Especie", "Lider", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Especie": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_merval = pd.concat([df_merval, new_row], ignore_index=True)

#print("Lo mejor")
df_merval

23


,Especie,Lider,Valor,% dia,Fecha
0,Especie,NaN,▼,Último,2025-02-11
1,ALUA,NaN,"832,00","-4,59",2025-02-11
2,BBAR,NaN,"7.670,00","-6,46",2025-02-11
3,BMA,NaN,"10.775,00","-4,22",2025-02-11
4,BYMA,NaN,"433,50","-6,07",2025-02-11
5,CEPU,NaN,"1.520,00","-4,70",2025-02-11
6,COME,NaN,"199,75","-5,44",2025-02-11
7,CRES,NaN,"1.355,00","-3,90",2025-02-11
8,CVH,NaN,"6.710,00","-4,42",2025-02-11
9,EDN,NaN,"2.215,00","-4,94",2025-02-11


### MATBA FOREX

In [265]:
try:
    driver.quit()
except:
    print("no driver open")       

In [266]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://matbarofex.primary.ventures/")
time.sleep(15)

In [ ]:
body_xpath = '/html/body/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div[2]'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

In [ ]:
df_forex = pd.DataFrame(columns=[ "Instrumento", "Vol C", "Compra", "Venta", "Vol V","Fecha"])
for i in range(4):
    
    split_values = child_elements[i].text.split()

    new_row = pd.DataFrame([{
        "Instrumento": split_values[0],
        "Vol C": split_values[1],
        "Compra": split_values[2],
        "Venta": split_values[3],
        "Vol V": split_values[4],
        "Fecha": today
    }])
    #print(new_row)
    df_forex = pd.concat([df_forex, new_row], ignore_index=True)
df_forex

### BONISTAS

# ECONOMÍA Y FINANZAS USA  

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://www.bls.gov/news.release/cpi.nr0.htm")
time.sleep(15)

In [ ]:
text_box_xpath = '/html/body/div[2]/div[5]/div/div[1]/pre[1]'

# Wait until the element is present
text_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, text_box_xpath))
)

# Extract text from the specified box
extracted_text = text_box.text

# Print the extracted text
print(extracted_text)


In [ ]:
table_xpath = '/html/body/div[2]/div[5]/div/div[1]/table'

# Wait until the table element is present
table_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, table_xpath))
)

# Extract the HTML of the table
table_html = table_element.get_attribute('outerHTML')

# Read the table into a DataFrame using pandas
df_bls = pd.read_html(table_html)[0]  # Read first table found

# Display the DataFrame
df_bls


In [ ]:
driver.quit()

# Guardamos los resultados !

In [134]:
with pd.ExcelWriter(f'C:/Users/bd1266/OneDrive - Assurant, Inc/Documents/Projects/Python + Pywin32/Automation F/Results/Results_{today}.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    df_1.to_excel(writer, sheet_name='Indec Ppales Indicadores', index=False)
    df_indicadores_ipc.to_excel(writer, sheet_name='IPC Indicadores', index=False)
    national_data.to_excel(writer, sheet_name='IPC Nucleo x capitulos', index=False)
    national_data_variation.to_excel(writer, sheet_name='IPC Nucleo - Variacion', index=False)
    emae_df.to_excel(writer, sheet_name='EMAE', index=False)  
    df_4.to_excel(writer, sheet_name='PIB + Serie Des', index=False)
    pib_1.to_excel(writer, sheet_name='PIB Hist - Trimestrales', index=False)  
    pib_2.to_excel(writer, sheet_name='PIB Hist - precios corr', index=False)
    df_ppalesvariables.to_excel(writer, sheet_name='BCRA Variables', index=False)
    df_8.to_excel(writer, sheet_name='Rava Dolar', index=False)
    df_bonos.to_excel(writer, sheet_name='Rava Bonos', index=False)
    df_lopeor.to_excel(writer, sheet_name='Rava Lo Peor', index=False)
    df_lomejor.to_excel(writer, sheet_name='Rava Lo Mejor', index=False)
    df_merval.to_excel(writer, sheet_name='Acciones Merval', index=False)
    df_forex.to_excel(writer, sheet_name='Forex', index=False)
    df_bls.to_excel(writer, sheet_name='BLS')

In [180]:
extracted_content_emae_page3


'Cuentas nacionales Buenos Aires, 20 de diciembre de 2024\nFuente: INDEC, Dirección Nacional de Cuentas Nacionales.Estimación preliminar de octubre de 2024. Resumen ejecutivo\nEn octubre de 2024, el estimador mensual de actividad económica (EMAE) registró una\ncaída de 0,7% en la comparación interanual (ia) y un crecimiento de 0,6% respecto a septiembre en la medición desestacionalizada (s.e.).\nCon relación a igual mes de 2023, cinco sectores de actividad que conforman el EMAE\nregistraron subas en octubre, entre los que se destacan Explotación de minas y canteras (+7,4% ia) y Agricultura, ganadería, caza y silvicultura (+2,3% ia).\nLa actividad Explotación de minas y canteras (+7,4% ia) fue, a su vez, la de mayor\nincidencia positiva en la variación interanual del EMAE, seguida por Agricultura, ganadería, caza y silvicultura (+2,3% ia) e Intermediación ﬁnanciera (1,8% ia).\nPor su parte, diez sectores de actividad registraron caídas en la comparación interanual,\nentre los que se des

In [181]:
extracted_text_indec_ipc_

'Destacados del mes\nEl Nivel general del Índice de precios al consumidor registró un alza mensual de 2,7% en octubre de 2024, y acumuló en el\naño una variación de 107,0%. En la comparación interanual, el incremento alcanzó el 193,0%.\nLa división de mayor aumento en el mes fue Vivienda, agua, electricidad, gas y otros combustibles (5,4%) por las\nsubas en Alquiler de la vivienda y gastos conexos; Electricidad, gas y otros combustibles; y Suministro de agua, seguida de\nPrendas de vestir y calzado (4,4%).\nLa división con mayor incidencia fue Restaurantes y hoteles (4,3%) en las regiones GBA y Noroeste, mientras que en\nNoreste, Cuyo y Patagonia, fue Vivienda, agua, electricidad, gas y otros combustibles (5,4%). Por último, en Pampeana,\nla mayor incidencia se presentó en Alimentos y bebidas no alcohólicas (1,2%), donde se destacaron los aumentos de\nPan y Cereales; Carnes y derivados; y Frutas.\nLas dos divisiones que registraron las menores variaciones en octubre de 2024 fueron Tran

In [109]:
cuadro1_variaciones_porcentuales_periodo_anterior


,Año,Período,Sector_registrado_Sector_privado_registrado_igual_mes_del_año_anterior,Sector_registrado_Sector_privado_registrado_diciembre_del_año_anterior,Sector_registrado_Sector_público_igual_mes_del_año_anterior,Sector_registrado_Sector_público_diciembre_del_año_anterior,Sector_registrado_Total_registrado_(1)_igual_mes_del_año_anterior,Sector_registrado_Total_registrado_(1)_diciembre_del_año_anterior,Sector_privado_no_registrado_(2)_igual_mes_del_año_anterior,Sector_privado_no_registrado_(2)_diciembre_del_año_anterior,Total_índice_de_salarios_igual_mes_del_año_anterior,Total_índice_de_salarios_diciembre_del_año_anterior
7,2015,Octubre,///,73.97,///,75.24,///,74.44,…,…,…,…
8,2015,Noviembre,2.68,75.95,1.37,76.28,2.19,76.07,…,…,…,…
9,2015,Diciembre,1.86,77.36,1.18,77.18,1.6,77.29,…,…,…,…
10,2016,Enero,2.02,78.93,0.36,77.46,1.41,78.38,…,…,…,…
11,2016,Febrero,1.41,80.04,1.16,78.36,1.32,79.42,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
112,2024,Julio,6.27,5450.78,6.72,4390.43,6.42,5054.69,15.08,3040.48,7.47,4653.26
113,2024,Agosto,5.04,5725.38,4.65,4594.73,4.91,5303.03,10.58,3362.05,5.65,4916.19
114,2024,Septiembre,3.77,5941.03,3.92,4774.83,3.82,5505.4,10.35,3710.18,4.71,5147.61
115,2024,Octubre,3.95,6175.77,4.34,4982.02,4.08,5729.85,7.66,3994.36,4.59,5383.96


In [108]:
cuadro1_variaciones_porcentuales_periodo_anterior
cuadro2_variaciones_porcentuales_acumuladas

,Año,Período,Sector_registrado_Sector_privado_registrado_igual_mes_del_año_anterior,Sector_registrado_Sector_privado_registrado_diciembre_del_año_anterior,Sector_registrado_Sector_público_igual_mes_del_año_anterior,Sector_registrado_Sector_público_diciembre_del_año_anterior,Sector_registrado_Total_registrado_(1)_igual_mes_del_año_anterior,Sector_registrado_Total_registrado_(1)_diciembre_del_año_anterior,Sector_privado_no_registrado_(2)_igual_mes_del_año_anterior,Sector_privado_no_registrado_(2)_diciembre_del_año_anterior,Total_índice_de_salarios_igual_mes_del_año_anterior,Total_índice_de_salarios_diciembre_del_año_anterior
2,2016,Enero,///,2.02,///,0.36,///,1.41,…,…,…,…
3,2016,Febrero,///,3.47,///,1.53,///,2.74,…,…,…,…
4,2016,Marzo,///,4.44,///,8.58,///,5.99,…,…,…,…
5,2016,Abril,///,11.06,///,9.18,///,10.36,…,…,…,…
6,2016,Mayo,///,14.72,///,12.86,///,14.03,…,…,…,…
...,...,...,...,...,...,...,...,...,...,...,...,...
104,2024,Julio,235.06,105.78,169.96,84.94,210.74,98.52,178.86,95.44,206.19,98.11
105,2024,Agosto,224.85,116.14,166.63,93.54,203.41,108.27,184.1,116.12,200.62,109.31
106,2024,Septiembre,197.71,124.29,149.73,101.13,180.27,116.22,191.76,138.5,181.86,119.16
107,2024,Octubre,184.62,133.14,138.74,109.86,167.9,125.03,199.42,156.76,172.14,129.22
